# Some Differences Between Pandas And Spark Dataframes

*To launch a Spark-compatible notebook, you can run in a shell :*

*IPYTHON_OPTS="notebook" --packages com.databricks:spark-csv_2.10:1.0.3 pyspark*

In [1]:
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
sqlContext = SQLContext(sc)

## Reading

With Pandas, you easily read CSV file with read_csv().

CSV is not supported natively by Spark. You have to use a separate library : spark-csv.

In [3]:
pandasDF = pd.read_csv('train.csv')
sparkDF = sqlContext.read.format('com.databricks.spark.csv') \
                                .options(header='true').load('train.csv')

## Counting

*sparkDF.count()* and *pandasDF.count()* are not the exactly the same.

The first one returns the number of rows.

The second one returns the number of non NA/null observations for each column.

In [4]:
pandasDF.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [5]:
sparkDF.count()

891

*sparkDF.shape* does not exist.

In [6]:
print pandasDF.shape

(891, 12)


In [7]:
print sparkDF.shape()

AttributeError: 'DataFrame' object has no attribute 'shape'

## Viewing

Pandas *.head()* is beautiful. Spark *.head()* is ugly. Prefer *.show()*

In [8]:
pandasDF.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S


In [9]:
sparkDF.head(4)

[Row(PassengerId=u'1', Survived=u'0', Pclass=u'3', Name=u'Braund, Mr. Owen Harris', Sex=u'male', Age=u'22', SibSp=u'1', Parch=u'0', Ticket=u'A/5 21171', Fare=u'7.25', Cabin=u'', Embarked=u'S'),
 Row(PassengerId=u'2', Survived=u'1', Pclass=u'1', Name=u'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex=u'female', Age=u'38', SibSp=u'1', Parch=u'0', Ticket=u'PC 17599', Fare=u'71.2833', Cabin=u'C85', Embarked=u'C'),
 Row(PassengerId=u'3', Survived=u'1', Pclass=u'3', Name=u'Heikkinen, Miss. Laina', Sex=u'female', Age=u'26', SibSp=u'0', Parch=u'0', Ticket=u'STON/O2. 3101282', Fare=u'7.925', Cabin=u'', Embarked=u'S'),
 Row(PassengerId=u'4', Survived=u'1', Pclass=u'1', Name=u'Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex=u'female', Age=u'35', SibSp=u'1', Parch=u'0', Ticket=u'113803', Fare=u'53.1', Cabin=u'C123', Embarked=u'S')]

In [10]:
sparkDF.show(4)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25|     |       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925|     |       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+



*.tail()* exists with Pandas, but not with Spark

In [11]:
pandasDF.tail(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,NaN,Q


In [12]:
sparkDF.tail(5)

AttributeError: 'DataFrame' object has no attribute 'tail'

## Inferring types

With Pandas, you rarely have to bother with types : they are inferred. 

With Spark DataFrames, default types are assumed to be “strings”.

In [13]:
pandasDF.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [14]:
sparkDF.schema

StructType(List(StructField(PassengerId,StringType,true),StructField(Survived,StringType,true),StructField(Pclass,StringType,true),StructField(Name,StringType,true),StructField(Sex,StringType,true),StructField(Age,StringType,true),StructField(SibSp,StringType,true),StructField(Parch,StringType,true),StructField(Ticket,StringType,true),StructField(Fare,StringType,true),StructField(Cabin,StringType,true),StructField(Embarked,StringType,true)))

## Describing

Pandas and Spark *.describe()* give slightly different results for two reasons :

* In Pandas, NaN values are excluded. In Spark, NaN values make that computation of mean and standard deviation fail. 

* Standard deviation is not computed in the same way. Unbiased (or corrected) standard deviation by default in Pandas, and uncorrected standard deviation in Spark. The difference is the use of N-1 instead of N on the denominator.

In [15]:
pandasDF[['Survived', 'Age', 'Fare', 'Pclass']].describe()

,Survived,Age,Fare,Pclass
count,891.000000,714.000000,891.000000,891.000000
mean,0.383838,29.699118,32.204208,2.308642
std,0.486592,14.526497,49.693429,0.836071
min,0.000000,0.420000,0.000000,1.000000
25%,0.000000,20.125000,7.910400,2.000000
50%,0.000000,28.000000,14.454200,3.000000
75%,1.000000,38.000000,31.000000,3.000000
max,1.000000,80.000000,512.329200,3.000000


In [16]:
sqlContext.createDataFrame(pandasDF[['Survived', 'Age', 'Fare', 'Pclass']]).describe().show()

+-------+------------------+----+-----------------+------------------+
|summary|          Survived| Age|             Fare|            Pclass|
+-------+------------------+----+-----------------+------------------+
|  count|               891| 891|              891|               891|
|   mean|0.3838383838383838| NaN|32.20420796857464| 2.308641975308642|
| stddev|0.4863193178670999| NaN| 49.6655344447741|0.8356019334795164|
|    min|                 0|0.42|              0.0|                 1|
|    max|                 1|80.0|         512.3292|                 3|
+-------+------------------+----+-----------------+------------------+



In [17]:
# This doesn't work because Spark DataFrames did not infer numerical data types, see above
sparkDF.describe().show()

+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    max|
+-------+



## Wrangling

In Spark, you cannot create new columns with the '[ ]' operator. 

You have to use *.withColumn()*

In [18]:
pandasDF2 = pandasDF.copy()
pandasDF2['Fare2'] = pandasDF['Fare'] + 2

In [19]:
pandasDF2[['Fare', 'Fare2']].head(5)

,Fare,Fare2
0,7.2500,9.2500
1,71.2833,73.2833
2,7.9250,9.9250
3,53.1000,55.1000
4,8.0500,10.0500


In [20]:
sparkDF2 = sparkDF.withColumn('Fare2', sparkDF['Fare'] + 2)

In [21]:
sparkDF2[['Fare', 'Fare2']].show(5)

+-------+-------+
|   Fare|  Fare2|
+-------+-------+
|   7.25|   9.25|
|71.2833|73.2833|
|  7.925|  9.925|
|   53.1|   55.1|
|   8.05|  10.05|
+-------+-------+

